# zad 2 Jakub Ilińki

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List
import torch
import numpy as np
from torch.nn import functional as F

PAPUGA = 'flax-community/papuGaPT2'
DEVICE = "cuda"

tokenizer = AutoTokenizer.from_pretrained(PAPUGA)
model = AutoModelForCausalLM.from_pretrained(PAPUGA).to(DEVICE)
model.device

device(type='cuda', index=0)

In [38]:
class Sentence:
    def __init__(self, text: str):
        self.text = text
        self.prob = self.sentence_prob()
    
    @staticmethod 
    def log_probs_from_logits(logits, labels):
        logp = F.log_softmax(logits, dim=-1)
        logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
        return logp_label
    
    def sentence_prob(self):
        input_ids = tokenizer(self.text, return_tensors='pt')['input_ids'].to(DEVICE)
        
        with torch.no_grad():
            output = model(input_ids=input_ids)
            log_probs = self.log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
            seq_log_probs = torch.sum(log_probs)
            
        return seq_log_probs.cpu().numpy()
        
    def print(self) -> None:
        text = self.text  
        text = text.capitalize()
        text += '.'
        print(text)

In [39]:
from itertools import permutations

def generate_all_possible_sentences(text: str) -> List[str]: 
    words: List[str] = list(text.split())
    return [" ".join(p) for p in permutations(words)]

def get_k_most_propable(sentences: List[Sentence], k: int) -> List[Sentence]:
    sorted_sentences = sorted(sentences, key=lambda s: float(s.prob), reverse=True)
    return sorted_sentences[:k]

In [40]:
strings = [
    "babuleńka miała dwa rogate koziołki",
    "wiewiurki w parku zaczepiają przechodniów"
]

k = 5
for string in strings: 
    print()
    
    all_sequences = [Sentence(s) for s in generate_all_possible_sentences(string)]
    k_most_propable = get_k_most_propable(all_sequences, k)
    for s in k_most_propable:
        s.print()


Miała babuleńka dwa rogate koziołki.
Miała babuleńka dwa koziołki rogate.
Dwa rogate koziołki miała babuleńka.
Dwa babuleńka miała koziołki rogate.
Miała dwa koziołki rogate babuleńka.

Wiewiurki zaczepiają przechodniów w parku.
W parku wiewiurki zaczepiają przechodniów.
Wiewiurki zaczepiają w parku przechodniów.
Wiewiurki zaczepiają przechodniów parku w.
Wiewiurki w parku zaczepiają przechodniów.
